Ders 2'de bir convolutional sinir ağında (CNN) temel katmanın nasıl özellik çıkarımı yaptığı konusundaki tartışmamıza başlamıştık. Bu süreçteki ilk iki işlemin relu aktivasyonlu bir Conv2D katmanında nasıl gerçekleştiğini öğrenmiştik.

Bu derste, bu dizideki üçüncü (ve son) işleme bakacağız: maksimum pooling ile condense, ki bu Keras'ta bir MaxPool2D katmanı ile yapılır.

**Maksimum Havuzlama ile Yoğunlaştırma**

Önceki modelimize yoğunlaştırma adımını eklemek, bize şunu verecektir:

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(filters=64, kernel_size=3),
    layers.MaxPool2D(pool_size=2),
])

MaxPool2D katmanı, Conv2D katmanına çok benzer, ancak çekirdek (kernel) yerine basit bir maksimum fonksiyonu kullanır ve pool_size parametresi kernel_size'a benzer şekilde çalışır. Ancak, MaxPool2D katmanının evrişimli katmandaki çekirdekte olduğu gibi eğitilebilir ağırlıkları yoktur.

Geçen dersten çıkarım şekline tekrar bir göz atalım. MaxPool2D'nin Yoğunlaştırma (Condense) adımı olduğunu hatırlayın.

![](image10.png)

ReLU aktivasyonundan sonra, modelde sıfırlarla dolu "ölü alanlar" oluşabilir, yani bazı kısımlar hiçbir faydalı bilgi içermez. Bu alanları taşımak gereksiz yere modelin boyutunu büyütür.

Max pooling, bu durumu düzeltmek için kullanılır. Bir alanın en yüksek aktivasyonunu seçerek, sadece en önemli bilgiyi korur ve özellik haritasını daha küçük ve yoğun hale getirir. Bu, modelin daha verimli çalışmasını sağlar.

![](image11.png)

ReLU aktivasyonundan sonra uygulandığında, özellikleri "yoğunlaştırma" etkisi yapar. Havuzlama adımı, aktif piksellerin sıfır piksellere oranını artırır.

Let's add the "condense" step to the feature extraction we did in the example in Lesson 2. This next hidden cell will take us back to where we left off.

![](image12.png)

In [ ]:
import tensorflow as tf

image_condense = tf.nn.pool(
    input=image_detect, # image in the Detect step above
    window_shape=(2, 2),
    pooling_type='MAX',
    # we'll see what these do in the next lesson!
    strides=(2, 2),
    padding='SAME',
)


![image.png](image13.png)

Oldukça havalı! Umarım havuzlama adımının, görüntüyü en aktif pikseller etrafında yoğunlaştırarak özelliği nasıl yoğunlaştırabildiğini görebiliyorsunuzdur.

Sıfır piksellerini "önemsiz" olarak adlandırdık. Bu, hiç bilgi taşımadıkları anlamına mı geliyor? Aslında, sıfır pikselleri konum bilgisi taşır. Boş alan, özelliği görüntü içinde konumlandırmaya devam eder. MaxPool2D bu piksellerin bazılarını kaldırdığında, özellik haritasındaki bazı konum bilgilerini de kaldırmış olur. Bu, bir evrişimli sinir ağına (convnet) öteleme değişmezliği (translation invarience) adı verilen bir özellik kazandırır. Bu, maksimum pooling kullanan bir convnet'in, özellikleri görüntüdeki konumlarına göre ayırt etme eğiliminde olmayacağı anlamına gelir. ("Öteleme", matematiksel olarak bir şeyin konumunu döndürmeden veya şeklini ya da boyutunu değiştirmeden değiştirme anlamına gelir.)

Aşağıdaki özellik haritasına tekrar tekrar maksimum havuzlama uyguladığımızda ne olduğunu izleyin.

![](image14.png)

Orijinal görüntüdeki iki nokta, tekrarlanan havuzlama işleminden sonra ayırt edilemez hale geldi. Başka bir deyişle, havuzlama işlemi onların bazı konum bilgilerini yok etti. Ağ artık özellik haritalarında bu noktalar arasında ayrım yapamadığı için, orijinal görüntüde de aralarında ayrım yapamaz: bu konum farkına karşı değişmez (invariant) hale gelmiştir.

Aslında, pooling işlemi bir ağda sadece küçük mesafeler üzerinde translation invarience oluşturur, tıpkı görüntüdeki iki nokta örneğinde olduğu gibi. Başlangıçta birbirinden uzak olan özellikler, havuzlama işleminden sonra da ayrı kalacaktır; konum bilgisinin sadece bir kısmı kaybedilmiştir, tamamı değil.

![](image15.png)

Bu, özelliklerin konumlarındaki küçük farklılıklara karşı invariance , bir görüntü sınıflandırıcısı için sahip olunması gereken güzel bir özelliktir. Sadece perspektif veya çerçeveleme (framing) farklılıkları nedeniyle, aynı tür özellik orijinal görüntünün çeşitli bölümlerinde konumlandırılmış olabilir, ancak yine de sınıflandırıcının bunların aynı olduğunu tanımasını isteriz. Bu invariance ağa yerleşik olduğundan, eğitim için çok daha az veri kullanmakla yetinebiliriz: artık bu farkı göz ardı etmeyi öğretmemiz gerekmiyor. Bu, evrişimli ağlara sadece yoğun katmanları olan bir ağa göre büyük bir verimlilik avantajı sağlar. (6. Derste Veri Artırma ile ücretsiz değişmezlik elde etmenin başka bir yolunu göreceksiniz!)

Sonuç
Bu derste, özellik çıkarmanın son adımını öğrendik: MaxPool2D ile condense(yoğunlaştırma). 4. Derste, evrişim ve havuzlama konusundaki tartışmamızı kayan pencereler ile bitireceğiz.